In [129]:
import pandas as pd
from tqdm import tqdm

In [130]:
puzzles_df = pd.read_csv('puzzles.csv')
puzzle_info_df = pd.read_csv('puzzle_info.csv')


In [134]:
puzzles_df.query('num_wildcards != 0').index

Index([ 31,  33,  38,  40,  64,  65,  73,  76,  80,  89,  96, 100, 101, 107,
       110, 114, 116, 122, 140, 143, 145, 156, 160, 163, 164, 165, 186, 196,
       203, 206, 208, 211, 215, 218, 219, 227, 234, 241, 244, 245, 247, 248,
       249, 256, 257, 264, 265, 267, 270, 276, 277, 285, 286, 287, 291, 298,
       309, 310, 319, 321, 328, 336, 337, 342, 351, 352, 353, 357, 358, 359,
       360, 370, 372, 373, 381, 383, 385, 386, 387, 388],
      dtype='int64')

In [5]:
allowed_moves = {}

for _, row in puzzle_info_df.iterrows():
    allowed_moves[row['puzzle_type']] = eval(row['allowed_moves'])

# example of parsed allowed moves
for move_name, move_permutation in allowed_moves['cube_2/2/2'].items():
    print(move_name, move_permutation)

f0 [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11, 12, 13, 14, 15, 16, 20, 18, 21, 10, 8, 22, 23]
f1 [18, 16, 2, 3, 4, 5, 6, 7, 8, 0, 10, 1, 13, 15, 12, 14, 22, 17, 23, 19, 20, 21, 11, 9]
r0 [0, 5, 2, 7, 4, 21, 6, 23, 10, 8, 11, 9, 3, 13, 1, 15, 16, 17, 18, 19, 20, 14, 22, 12]
r1 [4, 1, 6, 3, 20, 5, 22, 7, 8, 9, 10, 11, 12, 2, 14, 0, 17, 19, 16, 18, 15, 21, 13, 23]
d0 [0, 1, 2, 3, 4, 5, 18, 19, 8, 9, 6, 7, 12, 13, 10, 11, 16, 17, 14, 15, 22, 20, 23, 21]
d1 [1, 3, 0, 2, 16, 17, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12, 13, 18, 19, 20, 21, 22, 23]


In [105]:
allowed_moves

{'cube_2/2/2': {'f0': [0,
   1,
   19,
   17,
   6,
   4,
   7,
   5,
   2,
   9,
   3,
   11,
   12,
   13,
   14,
   15,
   16,
   20,
   18,
   21,
   10,
   8,
   22,
   23],
  'f1': [18,
   16,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   0,
   10,
   1,
   13,
   15,
   12,
   14,
   22,
   17,
   23,
   19,
   20,
   21,
   11,
   9],
  'r0': [0,
   5,
   2,
   7,
   4,
   21,
   6,
   23,
   10,
   8,
   11,
   9,
   3,
   13,
   1,
   15,
   16,
   17,
   18,
   19,
   20,
   14,
   22,
   12],
  'r1': [4,
   1,
   6,
   3,
   20,
   5,
   22,
   7,
   8,
   9,
   10,
   11,
   12,
   2,
   14,
   0,
   17,
   19,
   16,
   18,
   15,
   21,
   13,
   23],
  'd0': [0,
   1,
   2,
   3,
   4,
   5,
   18,
   19,
   8,
   9,
   6,
   7,
   12,
   13,
   10,
   11,
   16,
   17,
   14,
   15,
   22,
   20,
   23,
   21],
  'd1': [1,
   3,
   0,
   2,
   16,
   17,
   6,
   7,
   4,
   5,
   10,
   11,
   8,
   9,
   14,
   15,
   12,
   13,
   18,
   19,
   20,
   21,
   22,
   2

In [106]:
# try to improve a better solution
# https://www.kaggle.com/code/jazivxt/visualize-using-pythreejs
submission_df = pd.read_csv('submission.csv')
submission_df

,id,moves
0,0,r1.-f1
1,1,f0.r1.f1.-d0.-d0.-f0.-r0.f0.d0
2,2,-d1.-r0.f0.-r1.f1.d1.-r1.-f0.d1.f0.d1.d1
3,3,-f0.d0.-r0.f0.-d0.-r0.d0.-f0.-r0.-f0
4,4,-r1.-f0.d0.r0.-d1.-d1.r1.d1.f0.r1.-d1.-r1
...,...,...
393,393,f19.f21.-f39.f20.f2.-f5.f7.-r3.f55.-f12.f65.-f...
394,394,-f31.-f22.f16.-f17.-f13.-f24.-f14.f2.f21.f44.f...
395,395,-r0.-f42.-f8.f16.-f49.f14.-f1.f56.f26.f35.f62....
396,396,f25.-f29.f46.f49.-f8.f27.f26.-f20.f2.-f20.f6.f...


In [107]:
def move_state(state, move, moves_pool):
    
    if '-' in move:
        move = move[1:]
        rev = True
    else:
        rev = False

    move = moves_pool[move]

    if rev:
        new_state = state[:]
        for i in range(len(move)):
            new_state[move[i]] = state[i]
        state = new_state
    else:
        state = [state[idx] for idx in move]
    
    return state


In [108]:
submission_df

,id,moves
0,0,r1.-f1
1,1,f0.r1.f1.-d0.-d0.-f0.-r0.f0.d0
2,2,-d1.-r0.f0.-r1.f1.d1.-r1.-f0.d1.f0.d1.d1
3,3,-f0.d0.-r0.f0.-d0.-r0.d0.-f0.-r0.-f0
4,4,-r1.-f0.d0.r0.-d1.-d1.r1.d1.f0.r1.-d1.-r1
...,...,...
393,393,f19.f21.-f39.f20.f2.-f5.f7.-r3.f55.-f12.f65.-f...
394,394,-f31.-f22.f16.-f17.-f13.-f24.-f14.f2.f21.f44.f...
395,395,-r0.-f42.-f8.f16.-f49.f14.-f1.f56.f26.f35.f62....
396,396,f25.-f29.f46.f49.-f8.f27.f26.-f20.f2.-f20.f6.f...


In [109]:
state_to_idx

{'A;D;C;F;B;E;D;E;D;E;A;C;A;C;F;B;D;A;F;C;F;B;E;B': 0,
 'A;E;C;E;B;B;D;B;A;D;C;E;F;C;D;B;D;A;F;C;F;F;E;A': 1,
 'C;A;E;E;A;D;D;B;F;C;C;E;D;A;D;B;B;B;F;C;F;F;E;A': 2,
 'E;C;E;A;F;C;D;B;D;A;C;E;B;B;D;B;A;D;F;C;F;F;E;A': 3,
 'A;E;E;A;F;C;D;B;D;A;C;E;D;B;B;B;C;D;E;C;F;F;A;F': 4,
 'E;A;A;E;C;D;D;B;F;C;C;E;D;A;B;B;D;B;E;C;F;F;A;F': 5,
 'B;A;A;E;E;D;A;B;F;C;C;E;D;A;B;F;E;D;C;B;C;F;D;F': 6,
 'F;A;A;E;B;D;A;B;F;C;C;E;D;D;B;C;C;E;B;D;E;F;A;F': 7,
 'F;A;D;E;A;B;B;D;A;C;E;E;D;D;B;C;C;E;B;F;C;F;A;F': 8,
 'A;A;B;E;C;B;A;D;A;C;E;E;D;D;B;F;E;F;C;B;C;F;D;F': 9,
 'A;A;B;E;C;B;C;B;A;C;A;D;D;D;E;E;E;F;B;F;D;C;F;F': 10,
 'A;A;A;A;B;B;C;C;C;C;D;D;D;D;E;E;E;E;B;B;F;F;F;F': 11,
 'A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F': 12}

In [110]:
len(moves_pool)

6

In [111]:

def solve_puzzle(puzzle_id):
    
    current_puzzle = puzzles_df.loc[puzzle_id]
    puzzle_type = current_puzzle['puzzle_type']
    initial_state = current_puzzle['initial_state']
    solution_state = current_puzzle['solution_state']
    
    moves = submission_df.loc[puzzle_id]['moves'].split('.')
    moves_pool = allowed_moves[puzzle_type]
    
    state = initial_state.split(';')
    
    state_list = []
    state_list.append(state)
    
    for move in moves:
        state = move_state(state, move, moves_pool)
        state_list.append(state)
    
    state_to_idx = {}
    for idx in range(len(state_list)):
        state_to_idx[';'.join(state_list[idx])] = idx
    
    res = []
    idx = 0
    
    # ignore last state
    for curr_idx in tqdm(list(range(len(state_list)-1))):
        
        if curr_idx != idx:
            continue
        
        state = state_list[idx] # state_list[0] initial state
        
        new_idx = -1
        new_move = ''
        
        for move in moves_pool:
            for reversed_move in ['', '-']:
                move = reversed_move + move
                new_state = ';'.join(move_state(state, move, moves_pool))
                
                if new_state in state_to_idx:
                    tmp_idx = state_to_idx[new_state]

                    if tmp_idx > new_idx:
                        new_idx = tmp_idx
                        new_move = move
        
        idx = new_idx
        res.append(new_move)
            
    print('PUZZLE_ID:', puzzle_id)
    print('MOVES BEFORE:', len(state_list)-1)
    print('MOVES AFTER:', len(res))
    print()
    
    return '.'.join(res)

In [115]:
puzzles_df.loc[280]

id                                                              280
puzzle_type                                           cube_19/19/19
solution_state    A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;...
initial_state     D;E;E;D;E;B;E;E;A;C;F;A;F;B;F;C;A;C;B;F;C;A;D;...
num_wildcards                                                     0
Name: 280, dtype: object

In [122]:
moves_pool.keys()

dict_keys(['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'r0', 'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9', 'r10', 'r11', 'r12', 'r13', 'r14', 'r15', 'r16', 'r17', 'r18', 'd0', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'd9', 'd10', 'd11', 'd12', 'd13', 'd14', 'd15', 'd16', 'd17', 'd18'])

In [128]:
puzzle_id=80
current_puzzle = puzzles_df.loc[puzzle_id]
puzzle_type = current_puzzle['puzzle_type']
initial_state = current_puzzle['initial_state']
solution_state = current_puzzle['solution_state']

moves = submission_df.loc[puzzle_id]['moves'].split('.')
moves_pool = allowed_moves[puzzle_type]

state = initial_state.split(';')

state_list = []
state_list.append(state)

for move in moves:
    state = move_state(state, move, moves_pool)
    state_list.append(state)

state_to_idx = {}
for idx in range(len(state_list)):
    state_to_idx[';'.join(state_list[idx])] = idx

res = []
idx = 0

# ignore last state
for curr_idx in tqdm(list(range(len(state_list)-1))):
    
    if curr_idx != idx:
        continue
    
    state = state_list[idx] # state_list[0] initial state
    
    new_idx = -1
    new_move = ''
    
    for move in moves_pool:
        for reversed_move in ['', '-']:
            move = reversed_move + move
            new_state = ';'.join(move_state(state, move, moves_pool))
            
            if new_state in state_to_idx:
                tmp_idx = state_to_idx[new_state]

                if tmp_idx > new_idx:
                    new_idx = tmp_idx
                    new_move = move
    
    idx = new_idx
    res.append(new_move)
        
print('PUZZLE_ID:', puzzle_id)
print('MOVES BEFORE:', len(state_list)-1)
print('MOVES AFTER:', len(res))
print()











100%|██████████| 32/32 [00:00<00:00, 26199.05it/s]

PUZZLE_ID: 80
MOVES BEFORE: 32
MOVES AFTER: 32



In [112]:
for i in range(398):
    
    if i in [282, 283]:
        continue
        
    submission_df.loc[i, 'moves'] = solve_puzzle(i)









100%|██████████| 2/2 [00:00<?, ?it/s]


PUZZLE_ID: 0
MOVES BEFORE: 2
MOVES AFTER: 2











100%|██████████| 9/9 [00:00<?, ?it/s]


PUZZLE_ID: 1
MOVES BEFORE: 9
MOVES AFTER: 9











100%|██████████| 12/12 [00:00<?, ?it/s]


...

PUZZLE_ID: 269
MOVES BEFORE: 1340
MOVES AFTER: 1340











































































































100%|██████████| 1377/1377 [00:01<00:00, 1074.28it/s]


PUZZLE_ID: 270
MOVES BEFORE: 1377
MOVES AFTER: 1377



























































































 94%|█████████▍| 1270/1353 [00:01<00:00, 1162.20it/s]


KeyboardInterrupt: 

In [ ]:
submission_df.to_csv('submission.csv', index=False)
submission_df